from IPython.display import display, HTML

display(HTML(epl_data.text))

# Rate Limited Request (429 error)

We apologize, but you have triggered rate limiting by our cloud service provider.

This could be for one of many reasons.


- You accessed more than thirty pages in less than a minute.

- You have written a bot that accessed too many files too quickly.

- Multiple people are accessing the site at the same time via your IP address.

- You are an employee of ESPN.


### Generally, we block traffic that we think is from a bot for an hour and traffic that we think is a real user for five minutes.



In [1]:
#https://www.youtube.com/watch?v=Nt7WJa2iu0s
#https://www.youtube.com/watch?v=0irmDBWLrco
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time
from datetime import datetime

In [23]:
def team_url_getter(seasonend: int):
    '''
    seasonend = year the season ends in YYYY format as int
        e.g. season 2021-2022, seasonend = 2022
    '''
    if datetime.now().year == seasonend:
        standigs_url = 'https://fbref.com/en/comps/9/PRemier-League-Stats'
    elif datetime.now().year > seasonend:
        standigs_url = f'https://fbref.com/en/comps/9/{str(seasonend -1)}-{str(seasonend)}/{str(seasonend -1)}-{str(seasonend)}-Premier-League-Stats'
    else: 
        print('Your season is in the future, no data available')
        return
    epl_data = requests.get(standigs_url)
    epl_soup = BeautifulSoup(epl_data.text)
    stat_tbl = epl_soup.select('table.stats_table')[0]
    links = [link.get("href") for link in stat_tbl.find_all('a')]
    urls = [f"https://fbref.com{url}" for url in links if '/squads/' in url]
    return urls

def team_stats_extract(url):
    season_df = pd.DataFrame()
    counter = 0
    for team in url:  
        if counter == 10:
            time.sleep(6+0)
            counter = 0     
        print(team)
        team_url = team
        team_nm = team_url.split('/')[-1].replace('-Stats', '')
        team_data = requests.get(team_url)
        counter +=1
        matches = pd.read_html(StringIO(team_data.text), match = 'Scores & Fixtures')
        soupt = BeautifulSoup(team_data.text)
        linkst = [l.get("href") for l in soupt.find_all('a')]
        l = [f"https://fbref.com{l}" for l in linkst if l and  '/all_comps/shooting' in l]
        shots_data = requests.get(l[0])
        counter +=1
        shooting = pd.read_html(StringIO(shots_data.text), match = 'Shooting')[0]
        shooting.columns = shooting.columns.droplevel()
        try: 
            matches_df = matches[0].merge(shooting[["Date", "Comp", "Opponent", 'Sh', 'SoT', 'Dist', "FK", "PK", "PKatt" ]], on = ['Date', 'Comp', 'Opponent'], how = 'left')
            matches_df.columns = [col.upper() for col in matches_df.columns]
            print(matches_df.shape)
            matches_df.loc[:,'TEAM'] = team_nm
            season_df = pd.concat([season_df, matches_df], axis = 0)
        except ValueError:
            continue

    return season_df


In [3]:
# CURRENT AS OF April 2024
#2023-2024
standigs_url = 'https://fbref.com/en/comps/9/PRemier-League-Stats'
epl_data = requests.get(standigs_url)
soup = BeautifulSoup(epl_data.text)
stdgs_tbl = soup.select('table.stats_table')[0]
links = stdgs_tbl.find_all('a')
lnks = [l.get("href") for l in links]
team_urls = [f"https://fbref.com{l}" for l in lnks if '/squads/' in l]
team_urls

['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/s

In [5]:
PL24 = team_stats_extract(team_urls)
PL24.to_csv('PL24.csv', index = False)

https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
(52, 25)
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
(58, 25)
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
(57, 25)
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
(56, 25)
https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats
(41, 25)
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
(51, 25)
https://fbref.com/en/squads/19538871/Manchester-United-Stats
(52, 25)
https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats
(53, 25)
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
(51, 25)
https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats
(50, 25)
https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats
(45, 25)
https://fbref.com/en/squads/fd962109/Fulham-Stats
(46, 25)
https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats
(44, 25)
https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats
(42, 25)
https://fbref.com/en/squads/cd051869/Brentford-Stats
(42, 25)
http

In [8]:
#2022-2023
# from IPython.display import display, HTML
# display(HTML(epl_data.text))
standigs_url = 'https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats'
epl_data = requests.get(standigs_url)
soup = BeautifulSoup(epl_data.text)
stdgs_tbl = soup.select('table.stats_table')[0]
links = stdgs_tbl.find_all('a')
lnks = [l.get("href") for l in links]
team_urls = [f"https://fbref.com{l}" for l in lnks if '/squads/' in l]
team_urls


['https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/2022-2023/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/2022-2023/Manchester-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/2022-2023/Newcastle-United-Stats',
 'https://fbref.com/en/squads/822bd0ba/2022-2023/Liverpool-Stats',
 'https://fbref.com/en/squads/d07537b9/2022-2023/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8602292d/2022-2023/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/2022-2023/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/2022-2023/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/2022-2023/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/2022-2023/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2022-2023/Chelsea-Stats',
 'https://fbref.com/en/squads/8cec06e1/2022-2023/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/7c21e445/2022-2023/West-Ham-United-Stats'

In [9]:
PL23 = team_stats_extract(team_urls)
PL23.to_csv('PL23.csv', index = False)

https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats
(61, 25)
https://fbref.com/en/squads/18bb7c10/2022-2023/Arsenal-Stats
(49, 25)
https://fbref.com/en/squads/19538871/2022-2023/Manchester-United-Stats
(62, 25)
https://fbref.com/en/squads/b2b47a98/2022-2023/Newcastle-United-Stats
(46, 25)
https://fbref.com/en/squads/822bd0ba/2022-2023/Liverpool-Stats
(52, 25)
https://fbref.com/en/squads/d07537b9/2022-2023/Brighton-and-Hove-Albion-Stats
(46, 25)
https://fbref.com/en/squads/8602292d/2022-2023/Aston-Villa-Stats
(41, 25)
https://fbref.com/en/squads/361ca564/2022-2023/Tottenham-Hotspur-Stats
(50, 25)
https://fbref.com/en/squads/cd051869/2022-2023/Brentford-Stats
(41, 25)
https://fbref.com/en/squads/fd962109/2022-2023/Fulham-Stats
(44, 25)
https://fbref.com/en/squads/47c64c55/2022-2023/Crystal-Palace-Stats
(41, 25)
https://fbref.com/en/squads/cff3d9bb/2022-2023/Chelsea-Stats
(50, 25)
https://fbref.com/en/squads/8cec06e1/2022-2023/Wolverhampton-Wanderers-Stats
(44, 25)
https:

In [10]:
#2021-2022
# from IPython.display import display, HTML
# display(HTML(epl_data.text))
standigs_url = 'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats'
epl_data = requests.get(standigs_url)
soup = BeautifulSoup(epl_data.text)
stdgs_tbl = soup.select('table.stats_table')[0]
links = stdgs_tbl.find_all('a')
lnks = [l.get("href") for l in links]
team_urls = [f"https://fbref.com{l}" for l in lnks if '/squads/' in l]
PL22 = team_stats_extract(team_urls)
PL22.to_csv('PL22.csv', index = False)

https://fbref.com/en/squads/b8fd03ef/2021-2022/Manchester-City-Stats
(58, 25)
https://fbref.com/en/squads/822bd0ba/2021-2022/Liverpool-Stats
(63, 25)
https://fbref.com/en/squads/cff3d9bb/2021-2022/Chelsea-Stats
(61, 25)
https://fbref.com/en/squads/361ca564/2021-2022/Tottenham-Hotspur-Stats
(54, 25)
https://fbref.com/en/squads/18bb7c10/2021-2022/Arsenal-Stats
(45, 25)
https://fbref.com/en/squads/19538871/2021-2022/Manchester-United-Stats
(49, 25)
https://fbref.com/en/squads/7c21e445/2021-2022/West-Ham-United-Stats
(56, 25)
https://fbref.com/en/squads/a2d435b3/2021-2022/Leicester-City-Stats
(58, 25)
https://fbref.com/en/squads/d07537b9/2021-2022/Brighton-and-Hove-Albion-Stats
(43, 25)
https://fbref.com/en/squads/8cec06e1/2021-2022/Wolverhampton-Wanderers-Stats
(42, 25)
https://fbref.com/en/squads/b2b47a98/2021-2022/Newcastle-United-Stats
(40, 25)
https://fbref.com/en/squads/47c64c55/2021-2022/Crystal-Palace-Stats
(44, 25)
https://fbref.com/en/squads/cd051869/2021-2022/Brentford-Stats
(44

In [24]:
year = 2021
PL21 = team_stats_extract(team_url_getter(year))
PL21.to_csv('PL'+str(year)[-2:]+'.csv', index = False)

https://fbref.com/en/squads/b8fd03ef/2020-2021/Manchester-City-Stats
(61, 25)
https://fbref.com/en/squads/19538871/2020-2021/Manchester-United-Stats
(61, 25)
https://fbref.com/en/squads/822bd0ba/2020-2021/Liverpool-Stats
(53, 25)
https://fbref.com/en/squads/cff3d9bb/2020-2021/Chelsea-Stats
(59, 25)
https://fbref.com/en/squads/a2d435b3/2020-2021/Leicester-City-Stats
(53, 25)
https://fbref.com/en/squads/7c21e445/2020-2021/West-Ham-United-Stats
(44, 25)
https://fbref.com/en/squads/361ca564/2020-2021/Tottenham-Hotspur-Stats
(59, 25)
https://fbref.com/en/squads/18bb7c10/2020-2021/Arsenal-Stats
(58, 25)
https://fbref.com/en/squads/5bfb9659/2020-2021/Leeds-United-Stats
(40, 25)
https://fbref.com/en/squads/d3fd31cc/2020-2021/Everton-Stats
(46, 25)
https://fbref.com/en/squads/8602292d/2020-2021/Aston-Villa-Stats
(42, 25)
https://fbref.com/en/squads/b2b47a98/2020-2021/Newcastle-United-Stats
(43, 25)
https://fbref.com/en/squads/8cec06e1/2020-2021/Wolverhampton-Wanderers-Stats
(42, 25)
https://fbr